In [6]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [7]:
def write_string_to_file(filename, content):
    lines = content.split('\n')
    cleaned_lines = [line.lstrip("=-").strip() for line in lines]
    cleaned_content = '\n'.join(cleaned_lines)
    try:
        with open(filename, 'w') as file:
            file.write(cleaned_content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [8]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [9]:
usecase_name = "ViewDetailProduct"
project_name = "Shopee"
main_flow_prompt="""
Step 1: User clicks on a product card.
Step 2: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, shipping options, how many products left, information of the store sells that product, descriptions and ratings of users who have bought the product.
"""

alt_prompt_1="""
Main flow:
Step 1: User clicks on a product card.
Step 2: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, shipping options, how many products left, information of the store sells that product, descriptions and ratings of users who have bought the product.

Alternative flow 1: Product with many options 
At step 2 of the main flow: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, there are all options of that product with appropriate title (Ex: if the product is lipstick, the title of options should be color, if the product is cake, the title should be flavor) shipping options, how many products left, information of the store sells that product store sells that product, descriptions and ratings of users who have bought the product.
"""

alt_prompt_2="""
Main flow:
Step 1: User clicks on a product card.
Step 2: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, shipping options, how many products left, information of the store sells that product, descriptions and ratings of users who have bought the product.

Alternative flow 2: Product out of stock
At step 2 of the main flow: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, shipping options, how many products left, information of the store sells that product store sells that product, descriptions and ratings of users who have bought the product. On the picture of the product must have a black sticker with white note "Sold out".
"""

prompt_all="""
View detail information of a product

Objective: this use case allow users to view more information of a product including all options of that product, the store selling the product, shipping options, descriptions and the rating of users who have bought it. 
Actor: users

Main flow:
Step 1: User clicks on a product card.
Step 2: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, shipping options, how many products left, information of the store sells that product, descriptions and ratings of users who have bought the product.

Alternative flow 1: Product with many options 
At step 2 of the main flow: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, a picture of the product, there are all options of that product with appropriate title (Ex: if the product is lipstick, the title of options should be color, if the product is cake, the title should be flavor) shipping options, how many products left, information of the store sells that product store sells that product, descriptions and ratings of users who have bought the product.

Alternative flow 2: Product out of stock
At step 2 of the main flow: System navigates user to the product's detail page of the chosen product. There should be information about the product's name, shipping options, how many products left, information of the store sells that product store sells that product, descriptions and ratings of users who have bought the product. On the picture of the product must have a black sticker with white note "Sold out".
"""

In [10]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

condition_scenario_response = ""
if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)

filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response

promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response+ "\n" + sub2_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "1.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{'Options title': {'condition': {'valid': 'product has many options like color for lipstick or flavor for cake', 'invalid': 'product does not have options or not specified'}, 'type': 'text field'}}
Scenario 1: View Product Detail with Unspecified Options Title
Conditional scenario:
Scenario 1: View Product Detail with Unspecified Options Title
Flow scenario:
1. Successful Navigation to Product Detail Page Scenario
1. Product Has Multiple Options Scenario
1. Product Out of Stock Scenario
String has been written to ViewDetailProduct1.csv

Final result:1. View Product Detail with Unspecified Options Title
2. Successful Navigation to Product Detail Page Scenario
3. Product Has Multiple Options Scenario
4. Product Out of Stock Scenario


In [ ]:
print(main_gpt_response)
print("//")
print(sub1_gpt_response)
print("//")
print(sub2_gpt_response)

In [11]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)

filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response

promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response+ "\n" + sub2_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "1.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{}
Conditional scenario:
Scenario 1: View Product Detail with Unspecified Options Title
Flow scenario:
1. Successful Navigation to Product Detail Page
2. Product Detail Page Load Failure
1. Product Has Multiple Options Scenario
Product Out of Stock Scenario
String has been written to ViewDetailProduct1.csv

Final result:1. View Product Detail with Unspecified Options Title
2. Successful Navigation to Product Detail Page
3. Product Detail Page Load Failure
4. Product Has Multiple Options Scenario
5. Product Out of Stock Scenario


In [ ]:
print(main_gpt_response)
print("//")
print(sub1_gpt_response)
print("//")
print(sub2_gpt_response)

In [12]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)

filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response

promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response+ "\n" + sub2_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "1.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{'Options title': {'condition': {'valid': 'product has many options like color for lipstick or flavor for cake', 'invalid': 'product does not have options or not specified'}, 'type': 'text field'}}
Scenario 1: View Product Detail with Unspecified Options Title
Conditional scenario:
Scenario 1: View Product Detail with Unspecified Options Title
Flow scenario:
1. Successful Navigation to Product Detail Page Scenario
2. Product Out of Stock Scenario
3. Product Detail Page Load Failure Scenario
4. Product with No Ratings Scenario
1. Product with Multiple Options Scenario
1. Product Out of Stock Scenario
String has been written to ViewDetailProduct1.csv

Final result:1. View Product Detail with Unspecified Options Title
2. Successful Navigation to Product Detail Page Scenario
3. Product Out of Stock Scenario
4. Product Detail Page Load Failure Scenario
5. Product with No Ratings Scenario
6. Product with Multiple Options Scenario


In [13]:
print(main_gpt_response)
print("//")
print(sub1_gpt_response)
print("//")
print(sub2_gpt_response)

1. Successful Navigation to Product Detail Page Scenario
2. Product Out of Stock Scenario
3. Product Detail Page Load Failure Scenario
4. Product with No Ratings Scenario
//
1. Product with Multiple Options Scenario
//
1. Product Out of Stock Scenario


In [16]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

In [17]:
print(main_gpt_response)

1. Successful Navigation to Product Detail Page Scenario
2. Product Out of Stock Scenario
3. Product Detail Page Load Failure Scenario
4. Product with No Ratings Scenario
